In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 9.8 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 14.6 MB/s ta 0:00:01
     |████████████████████████████████| 303 kB 13.1 MB/s eta 0:00:01


     |████████████████████████████████| 1.4 MB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 12.5 MB/s eta 0:00:01     |█████████                       | 2.7 MB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 9.2 MB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 723 kB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 285 kB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 12.4 MB/s eta 0:00:01
     |██████████████████████████████

     |████████████████████████████████| 1.1 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 804.1 MB 12.3 MB/s eta 0:00:01   |▏                               | 5.3 MB 12.9 MB/s eta 0:01:03     |█████████                       | 228.7 MB 14.1 MB/s eta 0:00:41     |██████████                      | 251.8 MB 11.8 MB/s eta 0:00:47     |███████████████████▍            | 486.9 MB 14.4 MB/s eta 0:00:22     |████████████████████            | 499.8 MB 14.8 MB/s eta 0:00:21     |████████████████████████▏       | 606.1 MB 14.1 MB/s eta 0:00:15     |████████████████████████▍       | 613.0 MB 13.9 MB/s eta 0:00:14     |███████████████████████████▋    | 693.7 MB 12.2 MB/s eta 0:00:10     |████████████████████████████    | 706.3 MB 13.9 MB/s eta 0:00:08     |████████████████████████████▉   | 724.8 MB 13.4 MB/s eta 0:00:06     |█████████████████████████████▏  | 732.4 MB 12.1 MB/s eta 0:00:06     |█████████████████████

In [1]:
import tensorflow as tf
print(tf.__version__)

import os
from os import listdir
from transformers import ( 
    T5Tokenizer, 
    TFT5ForConditionalGeneration
)
import numpy as np
import time

import pickle

# import python files
import sys
sys.path.append("../..")

from python_files.evaluation.mover_score import MoverScore, MyMoverScore
from python_files.evaluation.rouge_score import RougeScore
from python_files.language_tokens import LanguageTokens
from python_files.model_loader import ModelLoader
from python_files.output_tools.latex_table_writer import LatexTableWriter
from python_files.tf_record_loader import TFRecordLoader
from python_files.summary_generator import SummaryGenerator
# from python_files.summary_data import SummaryData

2.4.1


In [2]:
!nvidia-smi

Tue Apr 20 11:44:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:08:00.0  On |                  N/A |
| 35%   45C    P0    53W / 260W |   1152MiB / 11016MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Params

In [3]:
model_size = "t5-base"

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

BATCH_SIZE = 8

root_folder = "../.."

result_folder = root_folder + "/results/"

## Language Tokens

In [4]:
tokenizer = T5Tokenizer.from_pretrained(model_size)
language_tokens = LanguageTokens(tokenizer, "tf")
prefix_size = language_tokens.prefix_size
language_token_order = ["de_de", "en_de", "de_en", "en_en"]

## Load data From Diretory

In [5]:
tf_record_loader = TFRecordLoader(root_folder, language_tokens, prefix_size, MAX_ARTICLE_LEN, MAX_HIGHLIGHT_LEN, BATCH_SIZE)

test_dataset = tf_record_loader.get_tfrecord_dataset("/data/cnn_daily_mail_test/")
test_dataset.prefetch(1024)

<PrefetchDataset shapes: ((None, 512), (None, 512), (None, 150), (None, 150), (None, 512), (None, 512), (None, 150), (None, 150), (None, 512), (None, 512), (None, 150), (None, 150), (None, 512), (None, 512), (None, 150), (None, 150)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>

## Model Loader

In [27]:
model_loader = ModelLoader(model_size, root_folder+"/models", "t5_cnn_daily_mail_plus_translation") 
# model_loader.yield_models()

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Score functions

In [6]:
mover_score = MoverScore()

def get_new_score_dicts():
    rouge_score_dict = {
        language_token_order[0]: RougeScore(),
        language_token_order[1]: RougeScore(),
        language_token_order[2]: RougeScore(),
        language_token_order[3]: RougeScore()
    }

    mover_score_dict = {
        language_token_order[0]: MyMoverScore(mover_score),
        language_token_order[1]: MyMoverScore(mover_score),
        language_token_order[2]: MyMoverScore(mover_score),
        language_token_order[3]: MyMoverScore(mover_score)
    }
    return rouge_score_dict, mover_score_dict
test = get_new_score_dicts()

In [7]:
def get_scores(data_dict, batch_size):
    results = []
    rouge_score_dict, mover_score_dict = get_new_score_dicts()
    for i, data_from_epoch in data_dict.items():
        mover_score_list = []
        batch_predicted = []
        batch_real = []
        
        j = 0
        for k, item in enumerate(data_from_epoch):
            rouge_score_dict[item['language_tag']](item['real_data'], item['pred_data'])
            mover_score_dict[item['language_tag']](item['real_data'], item['pred_data'])
            if (k % 100) == 0:
                print("k: {}".format(k))
            
        results.append((rouge_score_dict.copy(), mover_score_dict.copy()))
        rouge_score_dict, mover_score_dict = get_new_score_dicts()
        print("Epoch: {} done".format(i))
    return results

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
 
class MyScore:
    def __init__(self)-> None:
        super().__init__()
        self.de_de = []
        self.de_en = []
        self.en_de = []
        self.en_en = []
        self.mover_score_de_de = []
        self.mover_score_de_en = []
        self.mover_score_en_de = []
        self.mover_score_en_en = []

        self.score_order = ["rouge1", "rouge2", "rougeLsum"]
       
    def __call__(self, token, rouge_score, mover_score):
        if token == "de_de":
            self.de_de.append(np.array([rouge_score["rouge1"], rouge_score["rouge2"], rouge_score["rougeLsum"]]))
            self.mover_score_de_de.append(mover_score)
        elif token == "de_en":
            self.de_en.append(np.array([rouge_score["rouge1"], rouge_score["rouge2"], rouge_score["rougeLsum"]]))
            self.mover_score_de_en.append(mover_score)
        elif token == "en_de":
            self.en_de.append(np.array([rouge_score["rouge1"], rouge_score["rouge2"], rouge_score["rougeLsum"]]))
            self.mover_score_en_de.append(mover_score)
        elif token == "en_en":
            self.en_en.append(np.array([rouge_score["rouge1"], rouge_score["rouge2"], rouge_score["rougeLsum"]]))
            self.mover_score_en_en.append(mover_score)
        else:
            print("ERROR: token not known")
     
    @staticmethod
    def shift_array_right(x):
        zero_fill_up = np.zeros([1,x.shape[1]], dtype=np.float64)
        return np.concatenate([zero_fill_up, x])

    def plot_one(self, axs, title, arr):

        axs.set_title("ROUGE score: " + title)
        axs.set_ylim(0, 44)
        axs.set_xlim(1,10)
        
        arr = self.shift_array_right(arr)
        for j in range(3):
            axs.plot(arr[:,j], label=self.score_order[j])
        axs.legend()
 
    def plot(self):
        
        fig = plt.figure(constrained_layout=True, figsize=(12, 8))
        spec = plt.GridSpec(2, 3, figure=fig)
        f_ax1 = fig.add_subplot(spec[0, 0])
        f_ax2 = fig.add_subplot(spec[0, 1])
        f_ax3 = fig.add_subplot(spec[1, 0])
        f_ax4 = fig.add_subplot(spec[1, 1])
        self.plot_one(f_ax1, "de to de", np.array(self.de_de))
        self.plot_one(f_ax2, "de to en", np.array(self.de_en))
        self.plot_one(f_ax3, "en to de", np.array(self.en_de))
        self.plot_one(f_ax4, "en to en", np.array(self.en_en))
        
        f_ax5 = fig.add_subplot(spec[:, 2])
        f_ax5.set_title("Mover Score")
        f_ax5.set_xlim(1,10)
        f_ax5.plot(np.array([self.mover_score_de_de[0]] + self.mover_score_de_de), label="de to de")
        f_ax5.plot(np.array([self.mover_score_de_de[0]] + self.mover_score_de_en), label="de to en")
        f_ax5.plot(np.array([self.mover_score_de_de[0]] + self.mover_score_en_de), label="en to de")
        f_ax5.plot(np.array([self.mover_score_de_de[0]] + self.mover_score_en_en), label="en to en")
        f_ax5.legend()
        plt.show()
        

        print("rouge_score max: ", "de to de", np.argmax(np.array(self.de_de), axis=0))
        print("rouge_score max: ", "de to en", np.argmax(np.array(self.de_en), axis=0))
        print("rouge_score max: ", "en to de", np.argmax(np.array(self.en_de), axis=0))
        print("rouge_score max: ", "en to en", np.argmax(np.array(self.en_en), axis=0))
        
    @staticmethod
    def append_to_lists(list1, list2, list3, item):
        
        list1.append(item[0])
        list2.append(item[1])
        list3.append(item[2])
    
    @staticmethod
    def get_propotion_of_baseline(x, baseline): 
        print(baseline, x)
        propotion = x / baseline 
        print(propotion)
        ret_list = []
        for x_i, p in zip(x, propotion):
            ret_list.append("{:.2f} ({:.2f})".format(x_i, p))
        return ret_list
        
    def get_latex_table(self, table_name, caption):
        print(self.mover_score_de_de[0])
        print(self.de_de[0])
        latex_table_dict = {
            'Scores': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'MoverScore'],
            'German to German':  list(self.de_de[0]) + [self.mover_score_de_de[0]],
            'English to English':  list(self.en_en[0]) +  [self.mover_score_en_en[0]],
            'English to German': self.get_propotion_of_baseline(self.en_de[0], self.de_de[0]) + [self.mover_score_en_de[0]],
            'German to English': self.get_propotion_of_baseline(self.de_en[0], self.en_en[0]) + [self.mover_score_de_en[0]]
        }
        print(latex_table_dict)
        latex_table_writer = LatexTableWriter()
        return latex_table_writer.write_table(table_name, latex_table_dict, caption) 
     

# Generation

## Generate Summarize
- Generate 240 / epoch

In [10]:
result_folder = root_folder + "/results/"
result_path = result_folder + "cdm_trans_epochs/"

In [1]:
skip_till_already_done = 8

for i, model in enumerate(model_loader.yield_models()):
    print("Epoch: ", i)
    if i < skip_till_already_done:
        print("Already done.")
    else:
        summary_gen = SummaryGenerator(tokenizer, language_token_order, False, epoch=i)
        predictions = summary_gen.generate_summaries(model, test_dataset, result_path, break_point=30)

NameError: name 'model_loader' is not defined

# Evaluation

## Load data from pickle file

In [3]:
predictions_dict = {}
epoch_files = listdir(result_path)
epoch_files.sort()
for i, file_name in enumerate(epoch_files):
    with open(result_path + file_name, "rb") as file:
        predictions_dict[i] = pickle.load(file)

NameError: name 'listdir' is not defined

In [9]:
scores = get_scores(predictions_dict, 16)

NameError: name 'get_scores' is not defined

In [10]:
latex_table_writer = LatexTableWriter()

In [11]:
my_score = MyScore()
for i, (rouge_score, mover_score) in enumerate(scores):
    for token in language_token_order:
        print("[{}]----- {} -----".format(i, token))
        my_score(token, rouge_score[token].result(), mover_score[token].result())

    
my_score.plot()

NameError: name 'MyScore' is not defined

## Final Result

### Load Model best epochs

In [11]:
model = model_loader.load_epoch(14)
result_full_ds_path = result_folder + "cdm_plus_trans_full_ds_14/"

In [12]:
summary_gen = SummaryGenerator(tokenizer, language_token_order, True)

In [13]:
predictions = summary_gen.generate_summaries(model, test_dataset, result_full_ds_path, use_break_point=False)

1434


### Load Final result data

In [9]:
result_full_ds_path = result_folder + "cdm_plus_trans_full_ds_14/"
predictions_dict = {}
predictions = []

epoch_files = listdir(result_full_ds_path)
epoch_files.sort()
for i, file_name in enumerate(epoch_files):
    with open(result_full_ds_path + file_name, "rb") as file:
        [predictions.append(item) for item in pickle.load(file)]
        
predictions_dict[0] = predictions

In [10]:
scores = get_scores(predictions_dict, 16)

k: 0
k: 100
k: 200
k: 300
k: 400
k: 500
k: 600
k: 700
k: 800
k: 900
k: 1000
k: 1100
k: 1200
k: 1300
k: 1400
k: 1500
k: 1600
k: 1700
k: 1800
k: 1900
k: 2000
k: 2100
k: 2200
k: 2300
k: 2400
k: 2500
k: 2600
k: 2700
k: 2800
k: 2900
k: 3000
k: 3100
k: 3200
k: 3300
k: 3400
k: 3500
k: 3600
k: 3700
k: 3800
k: 3900
k: 4000
k: 4100
k: 4200
k: 4300
k: 4400
k: 4500
k: 4600
k: 4700
k: 4800
k: 4900
k: 5000
k: 5100
k: 5200
k: 5300
k: 5400
k: 5500
k: 5600
k: 5700
k: 5800
k: 5900
k: 6000
k: 6100
k: 6200
k: 6300
k: 6400
k: 6500
k: 6600
k: 6700
k: 6800
k: 6900
k: 7000
k: 7100
k: 7200
k: 7300
k: 7400
k: 7500
k: 7600
k: 7700
k: 7800
k: 7900
k: 8000
k: 8100
k: 8200
k: 8300
k: 8400
k: 8500
k: 8600
k: 8700
k: 8800
k: 8900
k: 9000
k: 9100
k: 9200
k: 9300
k: 9400
k: 9500
k: 9600
k: 9700
k: 9800
k: 9900
k: 10000
k: 10100
k: 10200
k: 10300
k: 10400
k: 10500
k: 10600
k: 10700
k: 10800
k: 10900
k: 11000
k: 11100
k: 11200
k: 11300
k: 11400
k: 11500
k: 11600
k: 11700
k: 11800
k: 11900
k: 12000
k: 12100
k: 12200
k: 12

In [11]:
my_score = MyScore()
for i, (rouge_score, mover_score) in enumerate(scores):
    for token in language_token_order:
        print("[{}]----- {} -----".format(i, token))
        my_score(token, rouge_score[token].result(), mover_score[token].result())

[0]----- de_de -----
rouge1 = 34.00, 95% confidence [33.74, 34.24]
rouge2 = 15.22, 95% confidence [15.01, 15.43]
rougeLsum = 24.59, 95% confidence [24.35, 24.79]
Moverscore: 0.56
[0]----- en_de -----
rouge1 = 34.09, 95% confidence [33.87, 34.31]
rouge2 = 14.86, 95% confidence [14.67, 15.04]
rougeLsum = 24.68, 95% confidence [24.48, 24.90]
Moverscore: 0.56
[0]----- de_en -----
rouge1 = 37.82, 95% confidence [37.62, 38.05]
rouge2 = 15.43, 95% confidence [15.25, 15.60]
rougeLsum = 35.15, 95% confidence [34.94, 35.34]
Moverscore: 0.56
[0]----- en_en -----
rouge1 = 41.13, 95% confidence [40.89, 41.37]
rouge2 = 19.71, 95% confidence [19.47, 19.97]
rougeLsum = 38.38, 95% confidence [38.15, 38.64]
Moverscore: 0.57


In [12]:
print(my_score.get_latex_table("Result scores of the 5th epoch test data.", "final_results_cdm"))

0.5627655611196217
[33.99786073 15.21584077 24.58561732]
[33.99786073 15.21584077 24.58561732] [34.09157427 14.85838715 24.67569465]
[1.00275645 0.9765078  1.00366382]
[41.1298469  19.70866718 38.37660779] [37.82158521 15.42716617 35.15012727]
[0.91956543 0.7827605  0.91592585]
{'Scores': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'MoverScore'], 'German to German': [33.997860727864136, 15.21584076927038, 24.585617316966985, 0.5627655611196217], 'English to English': [41.129846903016826, 19.708667177216316, 38.37660778640418, 0.5677933375829977], 'English to German': ['34.09 (1.00)', '14.86 (0.98)', '24.68 (1.00)', 0.5632629227124935], 'German to English': ['37.82 (0.92)', '15.43 (0.78)', '35.15 (0.92)', 0.5603824116855604]}
\begin{table}[htb]
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|}
			\hline
			Scores & German to German & English to English & English to German & German to English\\
			\hline\hline
			ROUGE-1 & 34.00 & 41.13 & 34.09 (1.00) & 37.82 (0.92)\\
			ROUGE-2 & 15.22 & 19.71 & 14.